In [1]:
import csv
import pandas as pd 
import sqlite3

# Step 1 Download the dataset and save it using Curl.

In [46]:
# Here I use the '-o' option to save the file with the name 'population.csv'
!curl -o population.csv "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/geonames-all-cities-with-a-population-1000/exports/csv?lang=en&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.8M    0 25.8M    0     0   997k      0 --:--:--  0:00:26 --:--:--  856k0k      0 --:--:--  0:00:15 --:--:-- 1025k


# Step 2 review the dataset


In [2]:
# First we open the csv file using the sep parameter since the file is separated
# by semicolon, not by comas
df = pd.read_csv('population.csv', sep = ';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140938 entries, 0 to 140937
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Geoname ID               140938 non-null  int64  
 1   Name                     140937 non-null  object 
 2   ASCII Name               140936 non-null  object 
 3   Alternate Names          116605 non-null  object 
 4   Feature Class            140938 non-null  object 
 5   Feature Code             140938 non-null  object 
 6   Country Code             140893 non-null  object 
 7   Country name EN          140770 non-null  object 
 8   Country Code 2           81 non-null      object 
 9   Admin1 Code              140911 non-null  object 
 10  Admin2 Code              121928 non-null  object 
 11  Admin3 Code              68829 non-null   object 
 12  Admin4 Code              20569 non-null   object 
 13  Population               140938 non-null  int64  
 14  Elev

In [3]:
# Since we are going to use SQLite and we can´t have spaces in the column names,
# we are going to rename the columns, replacing the spaces whith underscore and 
# convert the names into lower case 

df.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

# 

# Step 3 Create the DataBase


In [4]:
# For the creating the data base we need the name of the desire columns with their 
# respective data type. So I'm going to change the strings in the next dataframe 
# in order to have the respective datatypes for SQLite

df_types = pd.DataFrame(df.dtypes).reset_index().rename(columns= {'index':'column_name',0: 'type'})
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'INTEGER',
    'float64': 'REAL'
}

# Convert the column data types using replace()
df_types['type'] = df_types['type'].replace(dtype_mapping)

df_types



,column_name,type
0,geoname_id,INTEGER
1,name,TEXT
2,ascii_name,TEXT
3,alternate_names,TEXT
4,feature_class,TEXT
5,feature_code,TEXT
6,country_code,TEXT
7,country_name_en,TEXT
8,country_code_2,TEXT
9,admin1_code,TEXT


In [17]:
# Now we are going to write the query to create the db 

columns = ', '.join([f"{column} {data_type}" for column, data_type in
                      zip(df_types['column_name'], df_types['type'])])


conn = sqlite3.connect('geonames.db')
cursor = conn.cursor()

# Create a table in the database
cursor.execute('CREATE TABLE IF NOT EXISTS countries ({})'.format(columns))




# Step 4 insert the data into the table countries

In [18]:
df.to_sql('countries', conn, if_exists='replace', index=False)


140938

# Step 4 Retrieve the countries whithout megalopolies

In [19]:
# Execute the SQL query to find countries without cities with more than 10 million inhabitants
query = '''
    SELECT DISTINCT `country_name_en`
    FROM countries
    WHERE `country_name_en` NOT IN (
        SELECT DISTINCT `country_name_en`
        FROM countries
        WHERE Population > 10000000
    )
'''

# Fetch the results of the query
cursor = conn.execute(query)
results = cursor.fetchall()

# Close the database connection
conn.close()



# Step 6: Save the query result as a tab-separated value (TSV) file


In [21]:
output_file = 'countries_without_megapolis.tsv'
with open(output_file, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerow(['Country Name'])
    writer.writerows(results)

print(f"Result saved in {output_file}")

Result saved in countries_without_megapolis.tsv


### Just check how many countries doesn't have a megalopolies

In [24]:
no_megalopolies = pd.read_csv('countries_without_megapolis.tsv', sep='\t')
no_megalopolies

,Country Name
0,Saudi Arabia
1,Seychelles
2,"Sudan, The Republic of"
3,Sweden
4,Singapore
...,...
214,Northern Mariana Islands
215,Western Sahara
216,Niue
217,Turks and Caicos Islands


In [40]:
print(f"{len(no_megalopolies['Country Name'])} countries doesn't have a megalopoly")

219 countries doesn't have a megalopoly
